# Traducción a Python 
A continuación, en el archivo tiempo_respuesta.R se presenta una función de cálculo de tiempos de respuesta de solicitudes de clientes, esta función esta escrita en lenguaje R. Traduzca la función a lenguaje Python.

## Importar Librerias

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

# Conexion a Google Drive

In [3]:
#Ejecuta la celda y sigue los pasos, es necesario conectarse con la cuenta de usuario  en la que se encuentran los archivos.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Declarar directorios

In [4]:
source_Dir = '/content/drive/MyDrive/Desafío GM/data/Calendario.xlsx' # Ruta a archivo xlsx.

# Traducción R - Python

In [5]:
# Función para encontrar el nombre del día de ingreso y respuesta de la solicitud.
def get_name_of_day(x, calendario, date_generated, column_name):
  # Cambiar el nombre de las columnas con el número de índice.
  calendario.rename(columns={calendario.columns[0] : date_generated,
                  calendario.columns[1] : column_name}, inplace=True)
  #Conversión de columna de formato de objeto a fecha y hora.
  calendario[date_generated] = pd.to_datetime(calendario[date_generated], format="%d/%m/%Y")
  #Fusionar marcos de datos x y calendario por la columna "day_created".
  x = pd.merge(x,calendario, on=(date_generated) ,how='left')
  return x

In [6]:
# Encontrar la hora de inicio del dia laboral en el cual se genero el registro.
def get_hora_inicio_dia(DIA_AUX, DIA_SEMANA="Sábado"):
  #Posibles horas de inicio según el día.
  info_dict = {
    "Sábado": "09:30:00",
    "Domingo": "11:00:00",
  }
  DIA = DIA_AUX.strftime("%m/%d/%Y %H:%M:%S").split(' ')[0]
  #El horario por defecto corresponde a la entrada de un día hábil (08:30:00).
  HORA_COMIENZO = info_dict.get(DIA_SEMANA, "08:30:00")

  INICIO_CREATED = pd.to_datetime(f'{DIA} {HORA_COMIENZO}', format='%d/%m/%Y %H:%M:%S')
  return INICIO_CREATED

In [7]:
hora_termino_lab = datetime.strptime('21:00:00', "%H:%M:%S").time()

def one(y):
  tiempo_respuesta = pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).dt.total_seconds()
  return tiempo_respuesta

def two(y):
  tiempo_respuesta = pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_CREATED']).dt.total_seconds()
  return tiempo_respuesta

def three(y):
  tiempo_respuesta = pd.to_timedelta(y['UPDATEDON_AUX'] - y['UPDATEDON_AUX']).dt.total_seconds()
  return tiempo_respuesta

def four(y):  
  tiempo_respuesta = pd.to_timedelta(datetime.combine(date.today(), hora_termino_lab) - datetime.combine(date.today(), x['CREATEDON_AUX'][0].time())).total_seconds() + np.where(
      pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_ANSWERED']).total_seconds() < 0, 
           0, 
           pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_ANSWERED']).total_seconds())+ np.where(
               pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).total_seconds() > 1,
               pd.to_timedelta(y["UPDATEDON_AUX"] - y['CREATEDON_AUX']).total_seconds()*11.5*60*60,
                0)
  return tiempo_respuesta

def five(y):
  tiempo_respuesta = np.where(pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_ANSWERED']).dt.total_seconds() < 0, 
           0, 
           np.where(pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).dt.total_seconds() == 0,
                    0,
                     pd.to_timedelta(y['UPDATEDON_AUX'] - y['INICIO_ANSWERED']).dt.total_seconds()) + np.where(
                        pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).dt.total_seconds() > 1, pd.to_timedelta(y['UPDATEDON_AUX'] - y['CREATEDON_AUX']).dt.total_seconds()*11.5*60*60,0))
           
  return tiempo_respuesta


def perform_operation(y):

### SI ES EL MISMO DIA Y LA HORA DE CREACIÓN AJUSTADA ES MENOR A LA HORA DE INICIO TRABAJO, RESTO NORMAL 
  if (y["CREATEDON_AUX"] == y["UPDATEDON_AUX"] and y["INICIO_CREATED"] <= y["CREATEDON_AUX"] and y["CREATEDON_AUX"].time() < hora_termino_lab): 
    return one(y)

# SI ES EL MISMO DIA Y LA HORA DE CREACIÓN ES MENOR A LA HORA DE INICIO TRABAJO Y SE RESPONDE DESPUES DE LA HORA DE INGRESO, 
#RESTO DESDE LA HORA DE INICIO DEL DIA 
  if (y["CREATEDON_AUX"] == y["UPDATEDON_AUX"] and y["CREATEDON_AUX"] <= y["INICIO_CREATED"] and y["UPDATEDON_AUX"] > y["INICIO_CREATED"]): 
    return two(y)


# SI ES EL MISMO DIA Y LA HORA DE CREACIÓN ES MENOR A LA HORA DE INICIO TRABAJO Y SE RESPONDE ANTES DE LA HORA DE INGRESO, 
#RESTO NORMAL                         
  if (y["CREATEDON_AUX"] == y["UPDATEDON_AUX"] and y["CREATEDON_AUX"] <= y["INICIO_CREATED"] and y["UPDATEDON_AUX"] <= y["INICIO_CREATED"]):
    return three(y)                 
                         
                         
### SI ES OTRO DIA Y ANTES DE LA HORA DE TÉRMINO DEL DIA LABORAL, 
#RESTO EL TIEMPO REAL DE TRABAJO 
  if (y["CREATEDON_AUX"] <= y["UPDATEDON_AUX"] and y["CREATEDON_AUX"].time() <= hora_termino_lab): 
    return four(y) 


### SI ES OTRO DIA Y DESPUES DE LA HORA DE TÉRMINO DEL DIA LABORAL, RESTO EL TIEMPO REAL DE TRABAJO SOLO DEL OTRO DIA 
  if (y["CREATEDON_AUX"] <= y["UPDATEDON_AUX"] and y["CREATEDON_AUX"].time() > hora_termino_lab):
    return five(y)

In [9]:
# Crear Dataframe de prueba.
x = pd.DataFrame({"CREATEDON" : ["02/01/2017 09:06:10", "05/01/2017 09:06:10"],
                  "UPDATEDON" : ["07/01/2017 11:10:00", "09/01/2017 09:06:10"]})
def solicitud_time(x):
# Cambiar formato del registro a datetime.
  for register in x:
    x[register] = pd.to_datetime(x[register], format='%d/%m/%Y %H:%M:%S')

  x["CREATEDON_AUX"] = x["CREATEDON"] 
  x["UPDATEDON_AUX"] = x["UPDATEDON"] 

    #Obtener tiempo delta y transformarlo a segundos.
  x['TiempoRespuesta_2'] = pd.to_timedelta(x['UPDATEDON'] - x['CREATEDON']).dt.total_seconds()
  x['TiempoRespuesta_2']

    ###### 
  ## CALCULO REAL DEL TIEMPO DE SOLICITUDES
  # Obtiene las fechas de creacion y respuesta de consulta.
  x["day_created"]= x["CREATEDON_AUX"].dt.normalize()
  x["day_answer"] = x["UPDATEDON_AUX"].dt.normalize()

  #Obtener Dataframe con fechas y dias de la semana (por nombre) desde el calendario.
  calendario = pd.read_excel(source_Dir, usecols=[0,3])

  x = get_name_of_day(x, calendario, "day_created", "DIA_SEMANA_CREATED")
  x = get_name_of_day(x, calendario, "day_answer", "DIA_SEMANA_ANSWER")

  # Encontrar la hora de inicio del dia laboral en el cual se genero la peticion.
  x["INICIO_CREATED"] =  x.apply(lambda y: get_hora_inicio_dia(y['CREATEDON_AUX'], y['DIA_SEMANA_CREATED']), axis=1)
  # Encontrar la hora de inicio del dia laboral en el cual se genero la respuesta.
  x["INICIO_ANSWERED"] = x.apply(lambda y: get_hora_inicio_dia(y['UPDATEDON_AUX'], y['DIA_SEMANA_ANSWER']), axis=1)
  
  #Obtener tiempo de respuesta.
  x["tiempo_respuesta"] = x.apply(lambda y: perform_operation(y), axis=1)

  #Retorno del frame de datos con el tiempo de respuesta calculado.
  y = x[['CREATEDON', 'UPDATEDON', 'tiempo_respuesta']].copy()
  return y

In [12]:
# Probar la funcion.
y = solicitud_time(x)
y

ValueError: ignored